In [1]:
import os
import glob
from azureml.core import Workspace
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.data import DataType

In [2]:
# Hide cell for demo as we cannot (?) use db secrets here somehow
sp_auth = ServicePrincipalAuthentication(
  tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47",
  service_principal_id="219640dd-937c-44ee-b00e-01b319e97e24",
  service_principal_password="]N0Q685tdMFpUQ-pdyl.sZCHKfBE[1fW"
)

In [3]:
# Connect to the Azure Machine Learning Workspace
azureml_workspace = Workspace.from_config(auth=sp_auth)

dbfs_mnt_raw = "/mnt/raw/"
dbfs_mnt_processed = "/mnt/processed/"

In [4]:
# Read parquet files into Spark DataFrame
df = spark.read.parquet(dbfs_mnt_processed + '/redditcomments/')

# Get a sampled subset of data frame rows
sample_df = df.sample(False, 0.1)

display(sample_df)

text,id,meta,time,author,ups,downs,authorlinkkarma,authorkarma,authorisgold,subreddit
aw did they remove the part of irelia s lore that the body in the game is just a dummy and irelia s soul is bound in the weapons floating around it after she died,d00vn49,gaming,1.45556006E9,Rot1nPiecesOnTwitch,126.0,0.0,2089.0,3869.0,false,leagueoflegends
yorick,d01p3yv,gaming,1.4556096E9,KappaPride_PogChamp,11.0,0.0,1.0,35.0,false,leagueoflegends
i dont even know why fiora was staying mid when she had tp advantage,d027c9t,gaming,1.45564941E9,silentdemize,1.0,0.0,68.0,1989.0,false,leagueoflegends
we buffed kogmaw last patch just so we can nerf him this patch,d02aocz,gaming,1.45565389E9,artemasad,47.0,0.0,15729.0,12143.0,false,leagueoflegends
gt katarina gt never let passion interfere with duty the rumors says otherwise,d01dw1w,gaming,1.45558746E9,asianyeti,1.0,0.0,3938.0,17082.0,false,leagueoflegends
adding counterplay nerfing the wrong way give me my xerath stun back,d02i01k,gaming,1.45566374E9,ZetaZeta,7.0,0.0,1.0,6301.0,false,leagueoflegends
can anyone give me advice based on my naop http naopggsummonerusername 14firedog,d02awgc,gaming,1.45565414E9,14firedog,1.0,0.0,4.0,1304.0,false,leagueoflegends
the communication between millenium players was rly weird this series especially hans sama with the overextentions for farm and kaze with the 1v5 flanks,d02ctyr,gaming,1.4556567E9,SapphireHeaven,1.0,0.0,1.0,136.0,false,leagueoflegends
that s insane how did you block the minions with just one champ,d02pxqg,gaming,1.45567603E9,bobtheblob6,1.0,0.0,1.0,626.0,false,leagueoflegends
why would they nerf switness i get that its a strong item that almost everyone uses but would it be considered broken or something that the had to nerf it its not op or anything,d02evcx,gaming,1.45565939E9,Beejsbj,2.0,0.0,1.0,31.0,false,leagueoflegends


In [5]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

# MinMaxScaler Transformation
assembler = VectorAssembler(inputCols=["ups","downs","authorlinkkarma","authorkarma"], outputCol="vector").setParams(handleInvalid="skip")
scaler = MinMaxScaler(min=0.0, max=1.0,inputCol="vector", outputCol="vector_scaled")

pipeline = Pipeline(stages=[assembler, scaler])

scalerModel = pipeline.fit(df)
scaledData = scalerModel.transform(df)

#vector_scaled is out normalized data: 
display(scaledData.select("ups","downs","authorlinkkarma","authorkarma","vector","vector_scaled"))


ups,downs,authorlinkkarma,authorkarma,vector,vector_scaled
13.0,0.0,224.0,1588.0,"List(1, 4, List(), List(13.0, 0.0, 224.0, 1588.0))","List(1, 4, List(), List(0.01703255109765329, 0.5, 3.3793467602021574E-4, 0.0024084111762834276))"
2.0,0.0,85.0,23856.0,"List(1, 4, List(), List(2.0, 0.0, 85.0, 23856.0))","List(1, 4, List(), List(0.012869038607115822, 0.5, 1.2823414045409973E-4, 0.03418003444256267))"
1.0,0.0,24.0,32322.0,"List(1, 4, List(), List(1.0, 0.0, 24.0, 32322.0))","List(1, 4, List(), List(0.012490537471612415, 0.5, 3.620728671645169E-5, 0.04625918670465717))"
2.0,0.0,15.0,192.0,"List(1, 4, List(), List(2.0, 0.0, 15.0, 192.0))","List(1, 4, List(), List(0.012869038607115822, 0.5, 2.2629554197782303E-5, 4.166208906841001E-4))"
3.0,0.0,1.0,293.0,"List(1, 4, List(), List(3.0, 0.0, 1.0, 293.0))","List(1, 4, List(), List(0.013247539742619227, 0.5, 1.5086369465188202E-6, 5.607260617768881E-4))"
97.0,0.0,700.0,58381.0,"List(1, 4, List(), List(97.0, 0.0, 700.0, 58381.0))","List(1, 4, List(), List(0.04882664647993944, 0.5, 0.0010560458625631741, 0.08343974763046869))"
10.0,0.0,1.0,73.0,"List(1, 4, List(), List(10.0, 0.0, 1.0, 73.0))","List(1, 4, List(), List(0.015897047691143074, 0.5, 1.5086369465188202E-6, 2.4683360989160723E-4))"
1.0,0.0,3455.0,8008.0,"List(1, 4, List(), List(1.0, 0.0, 3455.0, 8008.0))","List(1, 4, List(), List(0.012490537471612415, 0.5, 0.005212340650222524, 0.011568363635844806))"
118.0,0.0,0.0,0.0,"List(0, 4, List(0), List(118.0))","List(1, 4, List(), List(0.05677517032551098, 0.5, 0.0, 1.426783872205822E-4))"
1.0,0.0,28.0,6549.0,"List(1, 4, List(), List(1.0, 0.0, 28.0, 6549.0))","List(1, 4, List(), List(0.012490537471612415, 0.5, 4.224183450252697E-5, 0.009486685966296511))"


In [6]:
cov = sample_df.cov("authorkarma","ups")
print("covariance for columns: authorkarma and ups is: {}".format(cov))

In [7]:
# Visualize the overall categories in our dataset
df = spark.read.parquet(dbfs_mnt_processed + 'redditcomments/')

display(df.groupby("meta").count())

meta,count
television,2338
news,2297
lifestyle,2169
learning,1716
entertainment,2460
gaming,4552
humor,2468


In [8]:
# We would like to sample part of our dataset for training, where we'll train on subcategory level
df_gaming = df.where(df.meta == "gaming")

display(df_gaming.groupby("subreddit").count())

subreddit,count
pokemon,588
dota2,741
starcraft,378
leagueoflegends,1333
tf2,305
skyrim,247
gaming,700
minecraft,260


In [9]:
# Script: both Azure ML Datastores and DBFS mount the same storage container
# To simplify working with both, we retrieve relative paths for the DBFS mounted files

def match_pattern_on_storage(pattern):
  # Glob for files matching the given pattern
  dbfs_path = glob.glob(r"/dbfs/" + dbfs_mnt_processed + pattern)

  # Get the relative paths of the files to the root of the storage mount
  return [os.path.relpath(path, "/dbfs/" + dbfs_mnt_processed) for path in dbfs_path]

In [10]:
# Like the DBFS Mount, the Azure ML Datastore references the same `processed` container on Azure Storage
processed_ds = Datastore.get(azureml_workspace, 'datastoreprocessed')

# Define the Azure ML Datasets
# 1. With each version including new comments that arrived through streaming
comments_full_dataset = TabularDatasetFactory.from_parquet_files([
      DataPath(processed_ds, path) for path in match_pattern_on_storage(
        "redditcomments/*/*.parquet"
      )
])

# 2. A subset of comments in the gaming category
comments_subset_gaming_dataset = TabularDatasetFactory.from_parquet_files([
      DataPath(processed_ds, path) for path in match_pattern_on_storage(
        "redditcomments/subreddit=gaming/*.parquet"
      )
])

# Script: note that by creating a dataset, we create a reference to the data source location, along with a copy of its metadata. Data remains in the storage container.

In [11]:
# Register a new dataset version for reference during training
comments_full_dataset.register(
    azureml_workspace,
    name="redditcomments",
    create_new_version=True,
    description="The full dataset of comments"
)

comments_subset_gaming_dataset.register(
    azureml_workspace,
    name="redditcomments_gaming",
    create_new_version=True,
    description="The dataset with a sample of comments on the topic `gaming`"
)

Out[29]: {
 "source": [
 "('datastoreprocessed', 'redditcomments/subreddit=gaming/part-00000-504eb4c6-077f-47ac-9c04-fdb8b1057337.c000.parquet')",
 "('datastoreprocessed', 'redditcomments/subreddit=gaming/part-00000-ecb9727f-6e71-4709-8d3b-7dddfcc9d55a.c000.parquet')",
 "('datastoreprocessed', 'redditcomments/subreddit=gaming/part-00000-fd4ee9fd-0289-4862-87eb-48f2c2eb02f2.c000.parquet')"
 ],
 "definition": [
 "GetDatastoreFiles",
 "ReadParquetFile",
 "DropColumns"
 ],
 "registration": {
 "id": "6d30e7bd-46b2-49ce-9a70-6b9a8ded045f",
 "name": "redditcomments_gaming",
 "version": 1,
 "description": "The dataset with a sample of comments on the topic `gaming`",
 "workspace": "Workspace.create(name='ml-e2e-bigdata-mls', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='build-e2e-ml-bigdata')"
 }
}